In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils.individual import Individual
from utils.draw import draw
import utils.protectedOperators as pOps

In [ ]:
NUM_ISLANDS = 7
ISLAND_POPULATION = 125
GENERATIONS = 1000
MAX_DEPTH = 11
MUTATION_RATE = 0.3
TOURNAMENT_SIZE = 3
MIGRATION_INTERVAL = 150
MIGRATION_SIZE = 4
PRINT_INTERVAL = 100
INDIVIDUAL_ATTEMPTS = 10
TEST_TRAIN = 0.7
STAGNATION_THRESHOLD = 60

POPULATION_SIZE = NUM_ISLANDS * ISLAND_POPULATION

In [ ]:
def plot_fitness_trend(fitness_history):
    plt.figure(figsize=(10, 5))
    plt.plot(fitness_history, linestyle='-', color='blue', linewidth=2, label='Best Fitness')
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.title("Fitness Evolution Over Generations")
    plt.legend()
    plt.grid()
    plt.show()

def plot_predictions(y_test, y_pred, fitness):
    sorted_idx = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[sorted_idx]
    y_pred_sorted = np.array(y_pred)[sorted_idx]
    plt.figure(figsize=(10, 5))
    plt.plot(y_test_sorted, label="Truth", color="red", linewidth=2)
    plt.plot(y_pred_sorted, label="Prediction", color="blue", linewidth=1)
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.title(f"Actual vs Predicted - Fitness: {fitness:.4f}")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
class SymbolicRegressionMC:
    def __init__(self, xTrain, yTrain,
                 populationSize=POPULATION_SIZE,
                 generations=GENERATIONS,
                 maxDepth=MAX_DEPTH,
                 individualAttempts=INDIVIDUAL_ATTEMPTS,
                 tournamentSize=TOURNAMENT_SIZE,
                 numIslands=NUM_ISLANDS,
                 mutationRate=MUTATION_RATE,
                 migrationInterval=MIGRATION_INTERVAL,
                 migrationSize=MIGRATION_SIZE,
                 stagnationThreshold=STAGNATION_THRESHOLD,
                 printInterval=PRINT_INTERVAL,
                 refreshFraction=0.2,
                 mcBiasProb=0.6,
                 penaltyLambda=1e-3,
                 maxNodes=250,
                 earlyStopPatience=None):
        # Data / hyperparams
        self.xTrain = xTrain
        self.yTrain = yTrain
        self.populationSize = populationSize
        self.generations = generations
        self.maxDepth = maxDepth
        self.individualAttempts = individualAttempts
        self.tournamentSize = tournamentSize
        self.printInterval = printInterval

        # GP / island params
        self.numIslands = numIslands
        self.islandSize = max(2, self.populationSize // max(1, self.numIslands))
        self.mutationRate = mutationRate
        self.migrationInterval = migrationInterval
        self.migrationSize = migrationSize

        # Monte Carlo / stagnation
        self.stagnationThreshold = stagnationThreshold
        self.refreshFraction = refreshFraction
        self.mcBiasProb = mcBiasProb  # bias toward mutating elites

        # Other controls
        self.penaltyLambda = penaltyLambda   # complexity penalty factor
        self.MAX_NODES = maxNodes            # bloat control on recombined children
        self.earlyStopPatience = earlyStopPatience or int(self.generations * 0.25)

        # internals
        self.populations = []  # list of islands: each is list of Individuals
        self.bestIndividual = None
        self.bestFitness = float('inf')  # penalized fitness tracked
        self.fitnessHistory = []

    # ---------------------------
    # Initialization / helpers
    # ---------------------------
    def initializePopulation(self):
        """Initialize islands with random individuals (valid fitness only)."""
        self.populations = []
        for _ in range(self.numIslands):
            island = []
            while len(island) < self.islandSize:
                ind = Individual(self.maxDepth, self.xTrain, self.yTrain, self.individualAttempts)
                ind.generate()
                if np.isfinite(ind.fitness):
                    island.append(ind)
            self.populations.append(island)

    def _computePenalizedFitness(self, pop):
        """Attach penalized fitness (_pf) to each individual."""
        for ind in pop:
            try:
                size = ind.size()
            except Exception:
                size = 0
            ind._pf = ind.fitness + self.penaltyLambda * size

    def _clearPenalizedMarks(self, pop):
        for ind in pop:
            if hasattr(ind, "_pf"):
                del ind._pf

    # ---------------------------
    # Selection
    # ---------------------------
    def tournamentSelection(self, population, k=None):
        k = k or self.tournamentSize
        k = min(k, len(population))
        participants = random.sample(population, k)
        participants.sort(key=lambda ind: getattr(ind, "_pf", ind.fitness))
        weights = [k - i for i in range(k)]
        return random.choices(participants, weights=weights, k=1)[0]

    def monteCarloParent(self, population, temperature, sample_k=3):
        """
        Monte Carlo selection with normalized fitness within the sample.
        Uses penalized fitness (_pf) if available.
        """
        if temperature <= 0:
            return min(population, key=lambda ind: getattr(ind, "_pf", ind.fitness))

        sample_k = min(sample_k, len(population))
        sample = random.sample(population, sample_k)

        # collect penalized fitnesses
        fits = np.array([getattr(ind, "_pf", ind.fitness) for ind in sample], dtype=float)
        # normalize to [0,1] where 0 = best
        minf, maxf = fits.min(), fits.max()
        span = (maxf - minf) if maxf > minf else 1.0
        norm = (fits - minf) / span

        # candidate and reference
        idx_cand = random.randrange(len(sample))
        cand = sample[idx_cand]
        idxRef = int(np.argmin(norm))
        ref = sample[idxRef]

        delta = norm[idx_cand] - norm[idxRef]
        if delta <= 0:
            return cand
        # acceptance probability (temperature on normalized scale)
        try:
            acceptance = np.exp(-delta / temperature)
        except OverflowError:
            acceptance = 0.0
        if random.random() < acceptance:
            return cand
        return ref

    # ---------------------------
    # Migration & refresh
    # ---------------------------
    def migrate(self):
        for i in range(self.numIslands):
            source = self.populations[i]
            # choose direction with small probability reversed
            if random.random() < 0.25:
                targetIndex = (i - 1) % self.numIslands
            else:
                targetIndex = (i + 1) % self.numIslands
            target = self.populations[targetIndex]

            sortedSource = sorted(source, key=lambda ind: getattr(ind, "_pf", ind.fitness))
            numElite = max(1, self.migrationSize // 2)
            numRand = self.migrationSize - numElite
            elites = sortedSource[:numElite]
            innovators = random.sample(source, min(numRand, len(source)))
            migrants = elites + innovators

            worstTarget = sorted(target, key=lambda ind: getattr(ind, "_pf", ind.fitness), reverse=True)
            for m, w in zip(migrants, worstTarget):
                if getattr(m, "_pf", m.fitness) < getattr(w, "_pf", w.fitness):
                    idx = target.index(w)
                    target[idx] = m.clone()
            self.populations[targetIndex] = target

    def mcRefresh(self, island, dynamicFraction):
        """Replace worst individuals in island with biased new ones."""
        island.sort(key=lambda ind: getattr(ind, "_pf", ind.fitness))
        numReplace = int(len(island) * dynamicFraction)
        if numReplace <= 0:
            return island
        survivors = island[:-numReplace]
        newInds = []
        attempts = 0

        while len(newInds) < numReplace and attempts < max(500, numReplace * 10):
            attempts += 1
            if random.random() < self.mcBiasProb and len(survivors) > 0:
                donorPool = survivors[:max(1, len(survivors) // 4)]
                donor = random.choice(donorPool).clone()
                donor.mutate()
                if np.isfinite(donor.fitness):
                    newInds.append(donor)
            else:
                individual = Individual(self.maxDepth, self.xTrain, self.yTrain, self.individualAttempts)
                individual.generate()
                if np.isfinite(individual.fitness):
                    newInds.append(individual)

        # If still short, fill by mutating survivors
        while len(newInds) < numReplace:
            donor = random.choice(survivors).clone()
            donor.mutate()
            newInds.append(donor)

        newIsland = survivors + newInds
        return newIsland

    # ---------------------------
    # Evolution loop
    # ---------------------------
    def evolve(self):
        self.initializePopulation()
        stagnationCount = 0
        islandStagnation = [0] * self.numIslands

        T0, T_min, tempDecay = 1.0, 0.01, 0.98
        mutationRate, minMutRate, mutationDecay = 0.5, 0.2, 0.995
        penaltyLambda = 0.002
        temperature = T0

        for gen in tqdm(range(self.generations), desc="Generations"):
            for idx in range(self.numIslands):
                currentPop = self.populations[idx]
                for ind in currentPop:
                    ind.fitness = ind.fitness + penaltyLambda * ind.size()

                newPop = []
                elite = min(currentPop, key=lambda ind: ind.fitness)
                newPop.append(elite.clone())

                mcProb = max(0.05, 0.5 * (1 - gen / (self.generations * 0.7)))
                if islandStagnation[idx] > self.stagnationThreshold // 3:
                    mcProb *= 2.0

                while len(newPop) < self.islandSize:
                    if random.random() < mutationRate:
                        if random.random() < mcProb:
                            parent = self.monteCarloParent(currentPop, temperature, 4)
                        else:
                            parent = self.tournamentSelection(currentPop)
                        child = parent.clone()
                        child.mutate()
                        newPop.append(child)
                    else:
                        if random.random() < mcProb:
                            p1 = self.monteCarloParent(currentPop, temperature, 4)
                            p2 = self.monteCarloParent(currentPop, temperature, 4)
                        else:
                            p1 = self.tournamentSelection(currentPop)
                            p2 = self.tournamentSelection(currentPop)
                        if p1 is p2:
                            p2 = random.choice(currentPop)
                        c1, c2 = p1.crossover(p2)
                        newPop.extend([c1, c2])

                combined = currentPop + newPop
                combined.sort(key=lambda ind: ind.fitness)
                bestBefore = currentPop[0].fitness
                self.populations[idx] = combined[:self.islandSize]

                if self.populations[idx][0].fitness < bestBefore:
                    islandStagnation[idx] = 0
                else:
                    islandStagnation[idx] += 1

            if (gen + 1) % 100 == 0:
                self.migrate()

            improve = False
            for island in self.populations:
                bestIsland = min(island, key=lambda ind: ind.fitness)
                if bestIsland.fitness < self.bestFitness:
                    self.bestFitness = bestIsland.fitness
                    self.bestIndividual = bestIsland.clone()
                    improve = True

            self.fitnessHistory.append(self.bestFitness)

            if improve:
                stagnationCount = 0
            else:
                stagnationCount += 1

            if stagnationCount >= self.stagnationThreshold:
                print(f"Gen {gen+1} - Full stagnation: refreshing islands and reheating")
                for i in range(self.numIslands):
                    self.populations[i] = self.mcRefresh(self.populations[i], self.refreshFraction)
                    islandStagnation[i] = 0
                temperature, mutationRate, stagnationCount = T0, 0.5, 0

            temperature = max(T_min, temperature * tempDecay)
            mutationRate = max(minMutRate, mutationRate * mutationDecay)

            if (gen + 1) % self.printInterval == 0:
                print(f"Gen {gen+1} | Best fitness: {self.bestFitness:.6f} | "
                      f"Temp: {temperature:.3f} | MutRate: {mutationRate:.3f} | "
                      f"Formula: {self.bestIndividual}")

        return self.bestIndividual, self.fitnessHistory


In [ ]:
def runSymbolicRegressionMC(problemNum):
    problem = np.load(f'./data/problem_{problemNum}.npz')
    xData, yData = problem['x'], problem['y']
    perm = np.random.permutation(len(yData))
    xData = xData[:, perm]
    yData = yData[perm]

    trainSize = int(TEST_TRAIN * len(yData))
    xTrain, yTrain = xData[:, :trainSize], yData[:trainSize]
    xTest, yTest = xData[:, trainSize:], yData[trainSize:]

    mc = SymbolicRegressionMC(xTrain, yTrain)
    print(f"\nProblem {problemNum}")
    bestIndividual, history = mc.evolve()

    testIndividual = bestIndividual.clone()
    testIndividual.xTrain = xTest
    testIndividual.yTrain = yTest
    yPred = testIndividual.computeFitness(True)

    print("Best Formula:", bestIndividual)
    print("Train fitness:", bestIndividual.fitness)
    print("Test fitness:", testIndividual.fitness)

    #bestIndividual.plot()
    plot_fitness_trend(history)
    plot_predictions(yTest, yPred, testIndividual.fitness)

In [ ]:

np.seterr(all='ignore')
runSymbolicRegressionMC(7)

Generations:  68%|██████▊   | 677/1000 [43:34<20:47,  3.86s/it]


KeyboardInterrupt: 